In [1]:
import re

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('anime_series.csv')
df.head()

<IPython.core.display.Javascript object>

,Unnamed: 0,title,genres,description,poster,url
0,0,One Piece,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...","Gold Roger was known as the ""Pirate King,"" the...",https://img.flawlessfiles.com/_r/300x400/100/5...,https://aniwatch.to/one-piece-100
1,1,Chainsaw Man,"['Action', 'Adventure', 'Demons', 'Horror', 'S...","Denji is robbed of a normal teenage life, left...",https://img.flawlessfiles.com/_r/300x400/100/b...,https://aniwatch.to/chainsaw-man-17406
2,2,Demon Slayer: Kimetsu no Yaiba Swordsmith Vill...,"['Action', 'Adventure', 'Demons', 'Historical'...",It adapts the story from volume 12 (chapter 98...,https://img.flawlessfiles.com/_r/300x400/100/d...,https://aniwatch.to/demon-slayer-kimetsu-no-ya...
3,3,Attack on Titan,"['Action', 'Mystery', 'Drama', 'Fantasy', 'Sho...","Centuries ago, mankind was slaughtered to near...",https://img.flawlessfiles.com/_r/300x400/100/d...,https://aniwatch.to/attack-on-titan-112
4,4,Jujutsu Kaisen (TV),"['Action', 'Demons', 'Horror', 'School', 'Shou...",Idly indulging in baseless paranormal activiti...,https://img.flawlessfiles.com/_r/300x400/100/8...,https://aniwatch.to/jujutsu-kaisen-tv-534


In [3]:
df.shape

(5067, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5067 entries, 0 to 5066
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   5067 non-null   int64 
 1   title        5067 non-null   object
 2   genres       5067 non-null   object
 3   description  5067 non-null   object
 4   poster       5067 non-null   object
 5   url          5067 non-null   object
dtypes: int64(1), object(5)
memory usage: 237.6+ KB


In [5]:
df['url'][0]

'https://aniwatch.to/one-piece-100'

In [6]:
df['poster'][0]

'https://img.flawlessfiles.com/_r/300x400/100/54/90/5490cb32786d4f7fef0f40d7266df532/5490cb32786d4f7fef0f40d7266df532.jpg'

In [7]:
df.duplicated().sum()

0

In [8]:
df.isnull().sum()

Unnamed: 0     0
title          0
genres         0
description    0
poster         0
url            0
dtype: int64

In [9]:
df.drop(columns=['Unnamed: 0','poster','url'], inplace=True)

In [10]:
df.head()

,title,genres,description
0,One Piece,"['Action', 'Adventure', 'Comedy', 'Drama', 'Fa...","Gold Roger was known as the ""Pirate King,"" the..."
1,Chainsaw Man,"['Action', 'Adventure', 'Demons', 'Horror', 'S...","Denji is robbed of a normal teenage life, left..."
2,Demon Slayer: Kimetsu no Yaiba Swordsmith Vill...,"['Action', 'Adventure', 'Demons', 'Historical'...",It adapts the story from volume 12 (chapter 98...
3,Attack on Titan,"['Action', 'Mystery', 'Drama', 'Fantasy', 'Sho...","Centuries ago, mankind was slaughtered to near..."
4,Jujutsu Kaisen (TV),"['Action', 'Demons', 'Horror', 'School', 'Shou...",Idly indulging in baseless paranormal activiti...


In [11]:
df['description'][1]

"Denji is robbed of a normal teenage life, left with nothing but his deadbeat father's overwhelming debt. His only companion is his pet, the chainsaw devil Pochita, with whom he slays devils for money that inevitably ends up in the yakuza's pockets. All Denji can do is dream of a good, simple life: one with delicious food and a beautiful girlfriend by his side. But an act of greedy betrayal by the yakuza leads to Denji's brutal, untimely death, crushing all hope of him ever achieving happiness.\r\n\r\nRemarkably, an old contract allows Pochita to merge with the deceased Denji and bestow devil powers on him, changing him into a hybrid able to transform his body parts into chainsaws. Because Denji's new abilities pose a significant risk to society, the Public Safety Bureau's elite devil hunter Makima takes him in, letting him live as long as he obeys her command. Guided by the promise of a content life alongside an attractive woman, Denji devotes everything and fights with all his might 

In [12]:
df['genres'][0]

"['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy', 'Shounen', 'Super Power']"

In [13]:
df['genres'] = df['genres'].apply(lambda x:x.replace("'",''))
df['genres'] = df['genres'].apply(lambda x:x.replace(" ",''))

In [14]:
df['description'] = df['description'].apply(lambda x:x.replace("\n", ''))
df['description'] = df['description'].apply(lambda x:x.replace("\r", ''))

In [15]:
import string
strings = string.punctuation

In [16]:
def remove_punctuations(text):
    for string in strings:
        text = text.replace(string, '')
        return text

In [17]:
df['description'] = df['description'].apply(remove_punctuations)

In [18]:
df['description'] = df['description'] + df['genres']

In [19]:
df['description'] = df['description'].apply(lambda x:x.lower())

In [20]:
df.drop(columns=['genres'], inplace=True)

In [21]:
df

,title,description
0,One Piece,"gold roger was known as the ""pirate king,"" the..."
1,Chainsaw Man,"denji is robbed of a normal teenage life, left..."
2,Demon Slayer: Kimetsu no Yaiba Swordsmith Vill...,it adapts the story from volume 12 (chapter 98...
3,Attack on Titan,"centuries ago, mankind was slaughtered to near..."
4,Jujutsu Kaisen (TV),idly indulging in baseless paranormal activiti...
...,...,...
5062,Perman (1983),"mitsuo encountered superman, an alien who is a..."
5063,The Dining of Link Lee,"in beijing, a city full of opportunities and p..."
5064,Detective Conan: Love Story at Police Headquar...,the special contains the footage from episodes...
5065,BeyWarriors: Cyborg,the newest series of the beyblade saga is beyw...


In [22]:
ps = PorterStemmer()

def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

df['description'] = df['description'].apply(stem)

In [23]:
cv = CountVectorizer(stop_words='english')
vector = cv.fit_transform(df['description'])

similarity = cosine_similarity(vector)

In [32]:
def recommendations(series):
    
    # Get the index of the movie that matches.
    series_index = df[df['title'] == series].index[0]
    
    # Get the pairwsie similarity scores of all movies with that movie
    cosine = similarity[series_index]
    
    # Sort the movies on similarity scores
    similar = sorted(list(enumerate(cosine)), key=lambda x:x[1], reverse=True)
    
    # 10 most similiar movies
    series_list = similar[1:11]
    
    # similarity = cosine[movie_index]
    
    for i in series_list:
        print(df.iloc[i[0]].title)

In [33]:
recommendations('Dragon Ball Z')

Dragon Ball Z Kai
Dragon Ball Z: Summer Vacation Special
Dragon Ball Super
Dragon Ball Z Special 1: Bardock, The Father of Goku
Goku: Midnight Eye
Dragon Ball GT
Dragon Ball
Thousands of worlds
UFO Robo Grendizer
Cerberus


In [26]:
import pickle

In [27]:
pickle.dump(df, open('anime_series.pkl','wb'))

In [28]:
pickle.dump(df.to_dict(), open('anime_series_dict.pkl','wb'))

In [29]:
pickle.dump(similarity, open('anime_series_similarity.pkl','wb'))